In [1]:
import numpy as np
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np

n_labels = 6

GDFS_path = "/content/drive/My Drive/Colab Notebooks/fake_lstm/"

def sample(arr, labels, seq):
	sample_sentences = []
	sample_labels = []
	for i in range(len(arr)):
		tokens = len(arr[i])
		if tokens < seq:
			inst = np.array(arr[i],dtype=np.float32)
			zeros = np.zeros((seq-tokens, 60))
			inst = np.concatenate([inst, zeros], axis=0)
			sample_sentences.append(inst)
			sample_labels.append(labels[i])
	sample_sentences = np.array(sample_sentences, dtype=np.float32)
	sample_labels = np.array(sample_labels, dtype=np.int32)
	onehot_labels = np.eye(n_labels)[sample_labels]
	print(sample_sentences.shape)
	print(onehot_labels.shape)    
	return np.nan_to_num(sample_sentences,copy=False), onehot_labels


def load_data(type):
	train_sentences = np.load(GDFS_path+"train_"+type+"_glove_m.npy", allow_pickle=True)
	train_labels = np.load(GDFS_path+"train_"+type+"_labels_m.npy", allow_pickle=True)
	val_sentences = np.load(GDFS_path+"val_"+type+"_glove_m.npy", allow_pickle=True)
	val_labels = np.load(GDFS_path+"val_"+type+"_labels_m.npy", allow_pickle=True)
	test_sentences = np.load(GDFS_path+"test_"+type+"_glove_m.npy", allow_pickle=True)
	test_labels = np.load(GDFS_path+"test_"+type+"_labels_m.npy", allow_pickle=True)
	return train_sentences, train_labels, test_sentences, test_labels, val_sentences, val_labels


train_sentences, train_labels, test_sentences, test_labels, val_sentences, val_labels = load_data("binary")




seq_length = 200

epochs = 10

batch_size = 32

batches = train_labels.shape[0]//batch_size

test_batches = test_labels.shape[0]//batch_size

val_batches = val_labels.shape[0]//batch_size


train_vects, train_labels = sample(train_sentences, train_labels, seq_length)
test_vects, test_labels = sample(test_sentences, test_labels, seq_length)
val_vects, val_labels = sample(val_sentences, val_labels, seq_length)




x = tf.placeholder(tf.float32, [None, seq_length, 60], name="x")
y = tf.placeholder(tf.float32, [None, n_labels], name="y")




def self_attention(inp, nodes, name):
  with tf.variable_scope(name):
    att = tf.matmul(inp, inp, transpose_b=True)
    soft = tf.nn.softmax(att)
#     soft_s = tf.matrix_band_part(soft, soft.get_shape()[1], 0)
    res = tf.matmul(soft, inp)
    dense_in = tf.reshape(res, [batch_size*seq_length, inp.get_shape()[-1]])
    output = tf.nn.leaky_relu(tf.layers.dense(dense_in, nodes, name="dense_"+name))
    output_reshaped = tf.reshape(output, [batch_size, seq_length, nodes])
    return output_reshaped + inp

def dense_norm(inp, nodes, name):
  with tf.variable_scope(name):
    re = tf.reshape(inp, [batch_size*seq_length, inp.get_shape()[-1]])
    output = tf.nn.leaky_relu(tf.layers.dense(re, nodes, name="dn_"+name))
    output_reshaped = tf.reshape(output, [batch_size, seq_length, nodes])
    return tf.contrib.layers.layer_norm(output_reshaped)
  
def self_att_norm(inp, curr, nodes, name):
  sa = self_attention(inp, curr, name)
  dn = dense_norm(sa, nodes, name)
  return dn
  
  
att1 = self_att_norm(x, 60, 60, "att1")

att2 = self_att_norm(att1, 60, 32, "att2")

att3 = self_att_norm(att2, 32, 16, "att3")

att4 = self_att_norm(att3, 16, 8, "att4")

att5 = self_att_norm(att4, 8, 8, "att5")

output_final = tf.reshape(att5, [batch_size, seq_length*8])

fc1 = tf.nn.dropout(tf.layers.dense(output_final, units=256, activation=tf.nn.leaky_relu), keep_prob=0.8)

fc2 = tf.nn.dropout(tf.layers.dense(fc1, units=64, activation=tf.nn.leaky_relu), keep_prob=1)

logit = tf.squeeze(tf.layers.dense(fc1, units=n_labels, activation=None))

loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=y))

op = tf.train.AdamOptimizer(0.0001).minimize(loss)

pred = tf.argmax(tf.nn.sigmoid(logit), axis=1)

actual = tf.argmax(y, axis=1)

acc = tf.reduce_mean(tf.to_float(tf.equal(pred, actual)))



saver = tf.train.Saver()

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  val_accs = 0
  for ep in range(epochs):
    accs = []
    lss = []

    for batch in range(batches):
      random_indexes = np.random.choice(train_labels.shape[0], batch_size)
      x_batch = train_vects[random_indexes]
      y_batch = train_labels[random_indexes]
      fd = {x: x_batch, y: y_batch}
      _, ls, ac = sess.run([op, loss, acc], fd)
      accs.append(ac)
      lss.append(ls)
    print("epoch : ", ep, ", loss : ", np.mean(lss), ", acc : ", np.mean(accs))
    accs = []
    lss = []
    for batch in range(val_batches):
      random_indexes = np.random.choice(val_labels.shape[0], batch_size)
      x_batch = val_vects[random_indexes]
      y_batch = val_labels[random_indexes]
      fd = {x: x_batch, y: y_batch}
      ls, ac = sess.run([loss, acc], fd)
      accs.append(ac)
      lss.append(ls)
    print("val set | loss : ", np.mean(lss), ", acc : ", np.mean(accs))
    if np.mean(accs) >= val_accs:
      saver.save(sess, "./model_ckpt")
      val_accs = np.mean(accs)
  accs = []
  saver.restore(sess, "./model_ckpt")
  print("saved modes val acc : ", val_accs)
  for batch in range(test_batches):
    random_indexes = np.random.choice(test_labels.shape[0], batch_size)
    x_batch = test_vects[random_indexes]
    y_batch = test_labels[random_indexes]
    fd = {x: x_batch, y: y_batch}
    ls, ac = sess.run([loss, acc], fd)
    accs.append(ac)
  print("test set | acc : ", np.mean(accs))

(10143, 200, 60)
(10143, 6)
(1248, 200, 60)
(1248, 6)


W0805 12:34:43.785321 140267289311104 deprecation.py:323] From <ipython-input-2-60c449ea3ad7>:75: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0805 12:34:43.791492 140267289311104 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


(1274, 200, 60)
(1274, 6)


W0805 12:34:45.416728 140267289311104 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0805 12:34:45.978199 140267289311104 deprecation.py:506] From <ipython-input-2-60c449ea3ad7>:104: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0805 12:34:46.038048 140267289311104 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be

epoch :  0 , loss :  0.25640884 , acc :  0.54189456
val set | loss :  0.24103265 , acc :  0.559375
epoch :  1 , loss :  0.2309669 , acc :  0.5823242
val set | loss :  0.24133453 , acc :  0.5578125
epoch :  2 , loss :  0.22045195 , acc :  0.61552733
val set | loss :  0.23534496 , acc :  0.5257813
epoch :  3 , loss :  0.21961626 , acc :  0.6099609
val set | loss :  0.23099723 , acc :  0.571875
epoch :  4 , loss :  0.21888742 , acc :  0.60849607
val set | loss :  0.23073407 , acc :  0.5734375
epoch :  5 , loss :  0.21279731 , acc :  0.6347656
val set | loss :  0.22529826 , acc :  0.578125
epoch :  6 , loss :  0.21192713 , acc :  0.63242185
val set | loss :  0.23045549 , acc :  0.5570313
epoch :  7 , loss :  0.21352813 , acc :  0.6267578
val set | loss :  0.22890928 , acc :  0.5703125
epoch :  8 , loss :  0.20816681 , acc :  0.64394534
val set | loss :  0.22489782 , acc :  0.57109374
epoch :  9 , loss :  0.20542994 , acc :  0.64941406


W0805 12:44:54.273107 140267289311104 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


val set | loss :  0.23419552 , acc :  0.5796875
saved modes val acc :  0.5796875
test set | acc :  0.60096157
